In [3]:
import src.config as config

In [4]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

current_date=Timestamp('2025-06-18 09:00:00')


In [5]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # download 2 files from website
    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    rides = pd.concat([rides, rides_2])

    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [6]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

from_date=Timestamp('2025-05-21 09:00:00'), to_date_=Timestamp('2024-06-19 09:00:00')
File 2024-05 was already in local storage
File 2024-06 was already in local storage


In [7]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)
ts_data = ts_data.rename(columns={"rides": "rides_count"})

100%|██████████| 265/265 [00:01<00:00, 240.17it/s]


In [8]:
print(ts_data.columns)
print(ts_data.head())

Index(['pickup_hour', 'rides_count', 'pickup_location_id', 'rides_count '], dtype='object')
          pickup_hour  rides_count  pickup_location_id  rides_count 
0 2025-05-21 09:00:00          0.0                   1           NaN
1 2025-05-21 10:00:00          0.0                   1           NaN
2 2025-05-21 11:00:00          1.0                   1           NaN
3 2025-05-21 12:00:00          0.0                   1           NaN
4 2025-05-21 13:00:00          1.0                   1           NaN


In [2]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_ts'],
    event_time='pickup_ts',
)

NameError: name 'config' is not defined

In [8]:
feature_group.insert(ts_data, write_options={"wait_for_job": True})

Uploading Dataframe: 60.07% |██████    | Rows 106973/178080 | Elapsed Time: 00:23 | Remaining Time: 00:06%4|1750175276.577|FAIL|rdkafka#producer-1| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Connection setup timed out in state CONNECT (after 30029ms in state CONNECT)
Uploading Dataframe: 100.00% |██████████| Rows 178080/178080 | Elapsed Time: 00:36 | Remaining Time: 00:00
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/alex0002/Resources/jobs/time_series_hourly_feature_group_1_offline_fg_materialization/config_1750173892102) to trigger the materialization job again.


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)